# Part 03
## Optimize SVM Parameters (C and Gamma) on RBF Kernel

### Alunos: Camilla Rocha & Tácio Nery

In [1]:
# imports
import operator
import random
import numpy
import math
import copy

from deap import base, benchmarks, creator, tools, algorithms

from sklearn.svm import SVR
from sklearn.model_selection import ShuffleSplit, KFold
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.datasets import load_boston
from sklearn import metrics

import matplotlib.pyplot as plt

In [2]:
db = load_boston()

cmin = math.pow(2, -15)
cmax = math.pow(2, 3)
gmin = math.pow(2, -5)
gmax = math.pow(2, 15)

In [3]:
X = db.data
y = db.target

In [4]:
def evaluate(individual):
    c = individual[0]
    gamma = individual[1]
    model = SVR(C=c,gamma=gamma)
    
    scores = []
    support_vectors = []
    kfold = KFold(n_splits=3)
    for k, (train,test) in enumerate(kfold.split(X,y)):
        model.fit(X[train], y[train])
        score = model.score(X[test], y[test])
        scores.append(score)
        support_vectors.append(sum(model.n_support_))
    
    return numpy.mean(scores), numpy.mean(support_vectors),

def crossover(ind1,ind2):
    temp = copy.deepcopy(ind1)
    ind1[0] = ind2[0]
    ind2[0] = temp[0]
    return ind1,ind2,

def mutation(individual, indpb):
    mu_c = numpy.mean([cmin,cmax])
    mu_gamma = numpy.mean([gmin,gmax])
    sigma_c = numpy.std([cmin,cmax])
    sigma_gamma = numpy.std([gmin,gmax])
    #MU
    if random.random() > indpb:
        individual[0] += random.gauss(mu_c, sigma_c)
        if individual[0] < cmin:
            individual[0] = cmin
        elif individual[0] > cmax:
            individual[0] = cmax
            
    #GAMMA
    if random.random() > indpb:
        individual[1] += random.gauss(sigma_c, sigma_gamma)
        if individual[1] < gmin:
            individual[1] = gmin
        elif individual[1] > gmax:
            individual[1] = gmax
    return individual,

In [5]:
creator.create("FitnessGA", base.Fitness, weights=(1.0,-1.0,))
creator.create("Individual", list, fitness=creator.FitnessGA)
toolboxGA = base.Toolbox()
toolboxGA.register("attr_c", random.uniform, cmin, cmax)
toolboxGA.register("attr_gamma", random.uniform, gmin, gmax)
toolboxGA.register("individual", tools.initCycle, creator.Individual, (toolboxGA.attr_c, toolboxGA.attr_gamma), 1)
toolboxGA.register("population", tools.initRepeat, list, toolboxGA.individual)
toolboxGA.register("evaluate", evaluate)
toolboxGA.register("mate", crossover)
toolboxGA.register("mutate", mutation, indpb=0.25)
toolboxGA.register("select", tools.selNSGA2)

In [6]:
def plot_pareto(pareto):
    x_values = []
    y_values = []
    for individual in pareto:
        fitnesses = toolboxGA.evaluate(individual)
        x_values.append(fitnesses[0])
        y_values.append(fitnesses[1])
    
    plt.plot(x_values, y_values, '.')
    plt.ylabel('SUPPORT VECTORS')
    plt.xlabel('SCORE')
    plt.show()

In [7]:
def main():
    MU = 100
    LAMBDA = 100
    pop = toolboxGA.population(n=20)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    pop, log = algorithms.eaMuPlusLambda(population=pop, toolbox=toolboxGA, mu=MU, lambda_=LAMBDA, 
                                         cxpb=0.5, mutpb=0.5, ngen=20, stats=stats,
                                         halloffame=hof, verbose=True)
    return hof,log

In [ ]:
results = main()
plot_pareto(results[0])

gen	nevals	avg         	std        	min         	max        
0  	20    	-4.41624e+08	6.23796e+08	-1.90851e+09	4.75063e+08
1  	100   	-6.2111e+08 	7.39298e+08	-1.95455e+09	1.22023e+08
2  	100   	-7.66201e+08	8.80332e+08	-1.95455e+09	-0.556857  
3  	100   	-9.60742e+08	9.64035e+08	-1.95455e+09	-0.556857  
4  	100   	-9.54501e+08	9.6591e+08 	-1.95706e+09	-0.556857  
5  	100   	-9.52384e+08	9.63408e+08	-1.95706e+09	-0.556857  
6  	100   	-9.43007e+08	9.57231e+08	-1.95706e+09	-0.556857  
7  	100   	-9.0411e+08 	9.37675e+08	-2.12118e+09	-0.556857  
8  	100   	-8.47999e+08	9.08389e+08	-2.12118e+09	-0.556857  
9  	100   	-8.23482e+08	8.79713e+08	-2.12118e+09	-0.556857  
10 	100   	-8.18123e+08	8.71557e+08	-2.12118e+09	-0.556857  
11 	100   	-8.39839e+08	8.52334e+08	-2.12118e+09	-0.556857  
12 	100   	-8.85395e+08	8.94175e+08	-2.12118e+09	-0.556857  
13 	100   	-9.08225e+08	9.14235e+08	-2.12118e+09	-0.556857  
14 	100   	-9.09652e+08	9.17346e+08	-2.12118e+09	-0.556857  
15 	100   	-9.10322e+08	